In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# Choose your prompt
prompt = "Explain who you are"  # English example
prompt = "너의 소원을 말해봐"   # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)
print(input_ids)
output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128
)
print(tokenizer.decode(output[0]))

/data/ephemeral/home/sh/level2-mrc-nlp-08/.venv_sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.15s/it]


[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]너의 소원을 말해봐
[|assistant|]EXAONE 3.0 모델로서, 저의 주된 목적은 사용자에게 정확하고 유용한 정보를 제공하는 것입니다. 저는 다양한 질문에 답변하고, 문제를 해결하며, 학습과 연구를 돕기 위해 설계되었습니다. 제가 도울 수 있는 다른 방법이 있다면 언제든지 말씀해 주세요![|endofturn|]


In [4]:
from datasets import load_from_disk
dataset = load_from_disk("./resources/data/train_dataset/")
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [5]:
val_dataset[0]

{'title': '전효숙',
 'context': '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 'question': '처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 'id': 'mrc-0-003264',
 'answers': {'answer_start': [284], 'text': ['한보철강']},
 'document_id': 9027,
 '__index_level_0__': 2146}

In [14]:
# pretrained model 과 tokenizer를 불러오기
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./resources/checkpoint/kosquadv1/checkpoint-4200")
model = AutoModelForCausalLM.from_pretrained("./resources/checkpoint/kosquadv1/checkpoint-4200", device_map="auto", trust_remote_code=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [10]:
from datasets import Dataset
val_dataset = val_dataset.shuffle(seed=104)
answers = [answer['text'] for answer in val_dataset['answers']]
test = Dataset.from_dict({'context':val_dataset['context'][:10], 'question':val_dataset['question'][:10], 'answers':answers[:10]})

for data in test:
        prompt = f'''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
        [|user|] Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that "없음", don't try to make up an answer. Please answer in short answer. Keep the answer as concise.
        {data['context']}
        Question:{data['question']}'''

        input_ids = tokenizer(
            prompt,
            return_tensors="pt"
        )
        output = model.generate(
            input_ids['input_ids'],
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=25
        )
        print(f"pred : {tokenizer.decode(output[0])}")
        print(f"answer : {data['answers']}")
        print('---------------------------------------------------------------------')

tensor([[   420,    453,  47982,    453,    422,   2358,    937,  11522,    394,
           5746,   1932,   1005,   7401,  10680,   8385,    373,    619,  12913,
          19415,    375,    582,    361,    560,     27,    420,    453,  14719,
            453,    422,  10258,    629,   2824,  11144,    670,   4981,    681,
           6054,    629,   3511,    952,    629,   1604,    375,   2386,    904,
           2390,    368,    445,   1941,    629,   6054,    373,   1924,   2825,
            817,    738,   3170,   1040,   9417,   2390,    368,    445,   4515,
            681,   2087,   1292,    662,   6054,    375,   8871,   6054,    666,
           3640,   6054,    375,  20741,    629,   6054,    850,  53967,    375,
            560,     27,  13554,  35859,    732,   2620,    643,  25004,   6604,
           1548,   7937,   2062,   6604,    696, 102357,  50156,   6199,  74466,
            634,  44601,  13456,    773,   2957,   5623,   1058,    380,    382,
           1881,    634,   8

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
from datasets import Dataset
val_dataset = val_dataset.shuffle(seed=104)
answers = [answer['text'] for answer in val_dataset['answers']]
test = Dataset.from_dict({'context':val_dataset['context'][:10], 'question':val_dataset['context'][:10], 'answers':answers[:10]})

for data in test:
        prompt = f'''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
        [|user|] Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that "없음", don't try to make up an answer. Please answer in short answer. Keep the answer as concise.
        {data['context']}
        Question:{data['question']}
        [|assistant|] [|endofturn|]'''

        messages = [
            {"role": "system", 
            "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        '''input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )'''
        input_ids = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt"
        )

        output = model.generate(
            input_ids.to("cuda"),
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=128
        )
        print(f"pred : {tokenizer.decode(output[0])}")
        print(f"answer : {data['answers']}")
        print('---------------------------------------------------------------------')

In [13]:
model

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=1024, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): Module

In [15]:
EXAONE_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that "없음", don't try to make up an answer. Please answer in short answer. Keep the answer as concise.
{}
Question:{}'''

In [32]:
question = '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?'
context = '요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & 니피싱 철도의 설립 인가가 떨어졌을 때였다. 스카버러 남쪽과 코보콘크 마을을 잇는 철도 노선 공사가 시작되었고 1871년 6월에 억스브릿지까지 철도가 완공되었다. 이 노선은 이후 1871년 11월에 캐닝턴까지 연장되었고 1872년 11월에 코보콘크까지 완공되었다. \n\n이 노선의 철로 궤간은 1067mm로 협궤 노선이였다. 목재와 장작 수요가 선로 용량이 넘칠 정도로 많았지만 1870년대에 다른 철도와 마찬가지로 경제적 난황을 이기 못해 수익이 줄어들었다. 투자자들은 수익이 줄어든 이유로 철도가 협궤로 지어져서 다른 표준궤 노선과 연계하여 화물 운송을 할 수 없는 점을 들었다 \n\n1881년 7월, 미들랜드 철도가 토론토 & 니피싱 철도를 인수하였고 이 노선은 표준궤로 전환되었다. 미들랜드 철도의 영향은 스카버러에서도 볼 수 있는데 노선 동쪽을 따라 나란히 달리는 미들랜드 애비뉴가 이 철도 회사의 이름을 따서 지어졌다. 1884년, 미들랜드 철도는 그랜드 트렁크 철도에 인수되었고 이는 이후 캐나다 내셔널 철도에 다시 인수되었다. CN은 이 노선을 억스브릿지 선으로 명명하였다\n\n억스브릿지 선을 따라 달리는 여객 철도는 통근객을 위한 열차가 아니였다. 처음에는 론빌에 있는 미들랜드정션역과 토론토를 잇는 열차가 두 대씩 양방향으로 운행했고 한 대는 북동쪽으로 코보콘크까지 운행하였다. 토론토 북동쪽에 있던 시골 마을의 수요는 저조하기만 하였으며 20세기에 들어서서 도로 개량과 자동차 보편화로 쇠락의 길을 걷게 되었다\n\n1955년에는 코보콘크에서 억스브릿지까지 여객 열차 운행이 중단되었으며 30년 뒤 선로 또한 폐선되었다. 1963년에는 억스브릿지까지 이어지는 여객열차 운행이 중단되고 CN 열차는 토론토 유니언역에서 마컴으로 가는 열차를 5시 20분에 딱 한 대 운행하였다. 토론토로 돌아오는 열차는 존재하지 않았다'

In [33]:
promt = EXAONE_TEMP.format(context, question)
print(promt)

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that "없음", don't try to make up an answer.
요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & 니피싱 철도의 설립 인가가 떨어졌을 때였다. 스카버러 남쪽과 코보콘크 마을을 잇는 철도 노선 공사가 시작되었고 1871년 6월에 억스브릿지까지 철도가 완공되었다. 이 노선은 이후 1871년 11월에 캐닝턴까지 연장되었고 1872년 11월에 코보콘크까지 완공되었다. 

이 노선의 철로 궤간은 1067mm로 협궤 노선이였다. 목재와 장작 수요가 선로 용량이 넘칠 정도로 많았지만 1870년대에 다른 철도와 마찬가지로 경제적 난황을 이기 못해 수익이 줄어들었다. 투자자들은 수익이 줄어든 이유로 철도가 협궤로 지어져서 다른 표준궤 노선과 연계하여 화물 운송을 할 수 없는 점을 들었다 

1881년 7월, 미들랜드 철도가 토론토 & 니피싱 철도를 인수하였고 이 노선은 표준궤로 전환되었다. 미들랜드 철도의 영향은 스카버러에서도 볼 수 있는데 노선 동쪽을 따라 나란히 달리는 미들랜드 애비뉴가 이 철도 회사의 이름을 따서 지어졌다. 1884년, 미들랜드 철도는 그랜드 트렁크 철도에 인수되었고 이는 이후 캐나다 내셔널 철도에 다시 인수되었다. CN은 이 노선을 억스브릿지 선으로 명명하였다

억스브릿지 선을 따라 달리는 여객 철도는 통근객을 위한 열차가 아니였다. 처음에는 론빌에 있는 미들랜드정션역과 토론토를 잇는 열차가 두 대씩 양방향으로 운행했고 한 대는 북동쪽으로 코보콘크까지 운행하였다. 토론토 북동쪽에 있던 시골 마을의 수요는 저조하기만 하였으며 20세

In [34]:
input_ids = tokenizer(promt, return_tensors="pt")['input_ids']
input_ids.shape

torch.Size([1, 651])

In [48]:
output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    num_beams = 1,
    max_new_tokens=20
)
print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that "없음", don't try to make up an answer.
요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & 니피싱 철도의 설립 인가가 떨어졌을 때였다. 스카버러 남쪽과 코보콘크 마을을 잇는 철도 노선 공사가 시작되었고 1871년 6월에 억스브릿지까지 철도가 완공되었다. 이 노선은 이후 1871년 11월에 캐닝턴까지 연장되었고 1872년 11월에 코보콘크까지 완공되었다. 

이 노선의 철로 궤간은 1067mm로 협궤 노선이였다. 목재와 장작 수요가 선로 용량이 넘칠 정도로 많았지만 1870년대에 다른 철도와 마찬가지로 경제적 난황을 이기 못해 수익이 줄어들었다. 투자자들은 수익이 줄어든 이유로 철도가 협궤로 지어져서 다른 표준궤 노선과 연계하여 화물 운송을 할 수 없는 점을 들었다 

1881년 7월, 미들랜드 철도가 토론토 & 니피싱 철도를 인수하였고 이 노선은 표준궤로 전환되었다. 미들랜드 철도의 영향은 스카버러에서도 볼 수 있는데 노선 동쪽을 따라 나란히 달리는 미들랜드 애비뉴가 이 철도 회사의 이름을 따서 지어졌다. 1884년, 미들랜드 철도는 그랜드 트렁크 철도에 인수되었고 이는 이후 캐나다 내셔널 철도에 다시 인수되었다. CN은 이 노선을 억스브릿지 선으로 명명하였다

억스브릿지 선을 따라 달리는 여객 철도는 통근객을 위한 열차가 아니였다. 처음에는 론빌에 있는 미들랜드정션역과 토론토를 잇는 열차가 두 대씩 양방향으로 운행했고 한 대는 북동쪽으로 코보콘크까지 운행하였다. 토론토 북동쪽에 있던 시골 마을의 수요는 저조하기만 하였으며 20세